[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nishani53/Khanal_MSc_Thesis_MSU/blob/main/Khanal_MSU_project_forest_cover_area_1980s-2020s/Khanal_MSU_project_forest_cover_area_1980s-2020s.ipynb)


# 🌲 Forest Market Coverage & Forest Area Overlay Analysis

This analysis measures how much **actual forest area** is covered by the **procurement zones of different forest products** in Michigan, across multiple years. It also breaks down **how much forest land was retained, lost, or newly added to market coverage** between time periods, categorized by **forest type**.


---

## 📌 Objective

To accurately quantify how much forestland falls within the market zones of:
- 🌳 Hardwood Sawlogs
- 🌲 Softwood Sawlogs
- 🧻 Pulpwood
- 🔥 Biomass

This is done by overlaying **vector-based market coverage shapefiles** with a **classified forest type raster** and summing forested area in acres.

---

## 🗺️ Data Used

### 1. **Forest Market Coverage Shapefiles**
- Represent zones where mills could procure wood products in a given year.
- Separate shapefiles used for retained, lost, or added areas between decades.

### 2. **Forest Type Raster**
- A spatial raster where each pixel represents a specific forest type group (e.g., Maple/Beech/Birch, Aspen/Birch).
- Resolution: 30m or similar.
- Units: Coded numeric values mapped to forest type names.

---

## 🧠 Method Summary

1. 📐 **Reproject geometries** to ensure they match the raster CRS.
2. ✂️ **Clip the forest raster** using each polygon (e.g., retained market area).
3. 📊 **Count forest pixels** within each polygon by forest type.
4. 📏 **Convert pixel counts to area in acres**, and total them.
5. 🧾 **Print and record results**, including forest type breakdowns.

---

## 📦 Python Tools Used

- `rasterio` – for raster reading, masking, and metadata
- `fiona` – for shapefile access
- `shapely` – for geometry handling
- `pyproj` – for coordinate transformation
- `numpy` – for efficient pixel counting

---

## 📈 Output

For each polygon (e.g., retained area from 1985–2023 pulpwood market):
- Area by forest type (in acres)
- Total forest area
- Aggregated forest area across all polygons

Sample Output:


Raster Analysis

In [1]:
!pip install rasterio geopandas shapely
!pip install geopandas fiona
!pip install matplotlib-scalebar
!pip install gdown
!pip install pydrive
!pip install matplotlib-scalebar


# Biomass Area change between 1980s to 2020s

# 1985 - 2023

In [8]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj
import os

# === Path to the 2019 forest cover raster ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# === Paths to retained and added market shapefiles ===
shapefiles = {
    "Retained": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_retained_1985_2023.shp",
    "Added": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_added_1985_2023.shp"
}

# === Forest type mapping (FIA group codes) ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Open the forest raster ===
with rasterio.open(raster_path) as src:
    pixel_area = src.res[0] * abs(src.res[1]) * 0.000247105  # pixel area in acres
    nodata = src.nodata
    raster_crs = src.crs

    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    # === Process each shapefile (Retained and Added) ===
    for label, shp_path in shapefiles.items():
        print(f"\n🔍 Processing '{label}' zone...\n")

        total_area = 0.0

        with fiona.open(shp_path, 'r') as shapefile:
            shp_crs = shapefile.crs

            for feature in shapefile:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, out_transform = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} does not overlap the raster. Skipping...")
                        continue
                    else:
                        raise

                data = out_image[0]
                if nodata is not None:
                    valid_data = data[data != nodata]
                else:
                    valid_data = data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

                forest_type_areas_acres = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas_acres = {
                    name: area for name, area in forest_type_areas_acres.items() if name is not None
                }

                polygon_area = sum(forest_type_areas_acres.values())
                total_area += polygon_area

        print(f"✅ Total forest area in '{label}' zone: {total_area:.2f} acres")



🔍 Processing 'Retained' zone...

✅ Total forest area in 'Retained' zone: 4031116.53 acres

🔍 Processing 'Added' zone...

✅ Total forest area in 'Added' zone: 11087354.25 acres


In [9]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_lost_1985_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST biomass market (1985–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 2.00 acres

✅ Total forest area in LOST biomass market (1985–2023): 2.00 acres


# 1994 to 2023

In [10]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj
import os

# === Paths ===
base_dir = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP"
raster_path = os.path.join(base_dir, "forest_cover_2019.tif")

shapefiles = {
    "Market Added (1994–2023)": os.path.join(base_dir, "biomass_market_added_1994_2023.shp"),
    "Market Retained (1994–2023)": os.path.join(base_dir, "biomass_market_retained_1994_2023.shp")
}

# === Forest group mapping (FIA-type codes) ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Reprojection function ===
def reproject_geom(geom, src_crs, dst_crs):
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
    return mapping(transform(transformer, shape(geom)))

# === Open the raster once ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105
    nodata = src.nodata
    raster_crs = src.crs

    for label, shp_path in shapefiles.items():
        print(f"\n🔎 Processing: {label}")
        total_area = 0.0

        with fiona.open(shp_path, 'r') as shapefile:
            shp_crs = shapefile.crs

            for feature in shapefile:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} does not overlap the raster. Skipping...")
                        continue
                    else:
                        raise

                data = out_image[0]
                if nodata is not None:
                    data = data[data != nodata]

                unique_types, pixel_counts = np.unique(data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area_acres
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                polygon_area = sum(forest_type_areas.values())
                total_area += polygon_area

                print(f"  • Polygon {polygon_id} total: {polygon_area:.2f} acres")

        print(f"✅ Total forest area in {label}: {total_area:.2f} acres")



🔎 Processing: Market Added (1994–2023)
  • Polygon 0 total: 7134786.22 acres
✅ Total forest area in Market Added (1994–2023): 7134786.22 acres

🔎 Processing: Market Retained (1994–2023)
  • Polygon 0 total: 7983684.56 acres
✅ Total forest area in Market Retained (1994–2023): 7983684.56 acres


In [11]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === File Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_lost_1994_2023.shp"

# === Define forest land cover values (based on NLCD-like classification) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]  # Adjust if needed

# === Open raster ===
with rasterio.open(raster_path) as src:
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # Reprojection function
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    total_forest_area = 0.0

    # === Open shapefile ===
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]

            # Mask forest values
            if nodata is not None:
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            forest_area_acres = np.sum(forest_mask) * pixel_area
            total_forest_area += forest_area_acres

            print(f"📌 Polygon {polygon_id} has {forest_area_acres:.2f} acres of forest")

    print(f"\n🌲 Total forest area lost (1994 → 2023): {total_forest_area:.2f} acres")


📌 Polygon 0 has 39732.78 acres of forest

🌲 Total forest area lost (1994 → 2023): 39732.78 acres


# 2002 to 2023

In [12]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj
import os

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Shapefiles for added and retained
shapefiles = {
    "Added": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_added_2002_2023.shp",
    "Retained": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_retained_2002_2023.shp"
}

# Forest group codes (FIA-based)
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Open raster once ===
with rasterio.open(raster_path) as src:
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    # Loop over both shapefiles (added & retained)
    for label, shp_path in shapefiles.items():
        print(f"\n🔍 Analyzing Biomass Market {label} (2002–2023):")

        total_forest_area = 0.0

        with fiona.open(shp_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, out_transform = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped: does not overlap raster.")
                        continue
                    else:
                        raise

                data = out_image[0]
                if nodata is not None:
                    valid_data = data[data != nodata]
                else:
                    valid_data = data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    name: area for name, area in forest_type_areas.items() if name is not None
                }

                poly_area = sum(forest_type_areas.values())
                total_forest_area += poly_area

                print(f"📌 Polygon {polygon_id}: {poly_area:.2f} acres")

        print(f"✅ Total Forest Area ({label}): {total_forest_area:.2f} acres\n")



🔍 Analyzing Biomass Market Added (2002–2023):
📌 Polygon 0: 7559652.38 acres
✅ Total Forest Area (Added): 7559652.38 acres


🔍 Analyzing Biomass Market Retained (2002–2023):
📌 Polygon 0: 7558818.40 acres
✅ Total Forest Area (Retained): 7558818.40 acres



In [13]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === File Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_lost_2002_2023.shp"

# === Forest LULC values ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Raster processing ===
with rasterio.open(raster_path) as src:
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    total_forest_area_acres = 0.0

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject if needed
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]

            # Apply forest mask
            if nodata is not None:
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            forest_area = np.sum(forest_mask) * pixel_area
            total_forest_area_acres += forest_area

            print(f"Polygon {polygon_id} forest loss area: {forest_area:.2f} acres")

    print(f"\n✅ Total forest area lost in biomass market (2002–2023): {total_forest_area_acres:.2f} acres")


Polygon 0 forest loss area: 14469.43 acres

✅ Total forest area lost in biomass market (2002–2023): 14469.43 acres


# 2018 to 2023

In [14]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_added_2018_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_retained_2018_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Biomass Market Added 2018–2023")
compute_forest_area(retained_shp, raster_path, "Biomass Market Retained 2018–2023")



🔍 Processing Biomass Market Added 2018–2023
Polygon 0: 2245983.68 acres
✅ Total forest area in 'Biomass Market Added 2018–2023': 2245983.68 acres

🔍 Processing Biomass Market Retained 2018–2023
Polygon 0: 12872487.10 acres
✅ Total forest area in 'Biomass Market Retained 2018–2023': 12872487.10 acres


12872487.0971875

In [15]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
loss_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\biomass_market_lost_2018_2023.shp"

# === Forest class values from LULC (e.g., NLCD codes) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]  # Adjust if needed for your specific LULC raster

# === Compute forest area ===
with rasterio.open(raster_path) as src:
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    print(f"\n🔍 Processing Biomass Market Lost 2018–2023")

    total_area = 0.0

    with fiona.open(loss_shp, 'r') as shp:
        shp_crs = shp.crs

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                    continue
                else:
                    raise

            data = out_image[0]
            if nodata is not None:
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            area = np.sum(forest_mask) * pixel_area
            total_area += area

            print(f"Polygon {polygon_id}: {area:.2f} acres")

    print(f"✅ Total forest area lost (2018–2023): {total_area:.2f} acres")



🔍 Processing Biomass Market Lost 2018–2023
Polygon 0: 1969973.59 acres
✅ Total forest area lost (2018–2023): 1969973.59 acres


# Hardwood Sawlogs Market Change

# 1985 to 2023 

In [16]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Hardwood Sawlogs (1985–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_added_1985_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_retained_1985_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Hardwood Market Added 1985–2023")
compute_forest_area(retained_shp, raster_path, "Hardwood Market Retained 1985–2023")



🔍 Processing Hardwood Market Added 1985–2023
Polygon 0: 49590.88 acres
✅ Total forest area in 'Hardwood Market Added 1985–2023': 49590.88 acres

🔍 Processing Hardwood Market Retained 1985–2023
Polygon 0: 18721292.56 acres
✅ Total forest area in 'Hardwood Market Retained 1985–2023': 18721292.56 acres


18721292.5625

In [28]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_lost_1985_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **hardwood** market (1985–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 6.89 acres

✅ Total forest area in LOST **hardwood** market (1985–2023): 6.89 acres


# 1994 to 2023

In [17]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Hardwood Sawlogs (1994–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_added_1994_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_retained_1994_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Hardwood Market Added 1994–2023")
compute_forest_area(retained_shp, raster_path, "Hardwood Market Retained 1994–2023")



🔍 Processing Hardwood Market Added 1994–2023
Polygon 0: 818.54 acres
✅ Total forest area in 'Hardwood Market Added 1994–2023': 818.54 acres

🔍 Processing Hardwood Market Retained 1994–2023
Polygon 0: 18770064.91 acres
✅ Total forest area in 'Hardwood Market Retained 1994–2023': 18770064.91 acres


18770064.911875002

In [29]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_lost_1994_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **hardwood** market (1994–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 1570.99 acres

✅ Total forest area in LOST **hardwood** market (1994–2023): 1570.99 acres


# 2002 to 2023

In [18]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Hardwood Sawlogs (2002–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_added_2002_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_retained_2002_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Hardwood Market Added 2002–2023")
compute_forest_area(retained_shp, raster_path, "Hardwood Market Retained 2002–2023")



🔍 Processing Hardwood Market Added 2002–2023
Polygon 0: 818.54 acres
✅ Total forest area in 'Hardwood Market Added 2002–2023': 818.54 acres

🔍 Processing Hardwood Market Retained 2002–2023
Polygon 0: 18770064.91 acres
✅ Total forest area in 'Hardwood Market Retained 2002–2023': 18770064.91 acres


18770064.911875002

In [30]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_lost_2002_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **hardwood** market (2002–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 939.39 acres

✅ Total forest area in LOST **hardwood** market (2002–2023): 939.39 acres


# 2018 to 2023

In [19]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Hardwood Sawlogs (2018–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_added_2018_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_retained_2018_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Hardwood Market Added 2018–2023")
compute_forest_area(retained_shp, raster_path, "Hardwood Market Retained 2018–2023")



🔍 Processing Hardwood Market Added 2018–2023
Polygon 0: 3613.91 acres
✅ Total forest area in 'Hardwood Market Added 2018–2023': 3613.91 acres

🔍 Processing Hardwood Market Retained 2018–2023
Polygon 0: 18767269.54 acres
✅ Total forest area in 'Hardwood Market Retained 2018–2023': 18767269.54 acres


18767269.536562502

In [31]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\hardwood_market_lost_2018_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **hardwood** market (2018–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 15.12 acres

✅ Total forest area in LOST **hardwood** market (2018–2023): 15.12 acres


# Softwood Sawlogs 

# 1985 to 2023

In [20]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Softwood Sawlogs (1985–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_added_1985_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_retained_1985_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Softwood Market Added 1985–2023")
compute_forest_area(retained_shp, raster_path, "Softwood Market Retained 1985–2023")



🔍 Processing Softwood Market Added 1985–2023
Polygon 0: 595183.28 acres
✅ Total forest area in 'Softwood Market Added 1985–2023': 595183.28 acres

🔍 Processing Softwood Market Retained 1985–2023
Polygon 0: 18167885.47 acres
✅ Total forest area in 'Softwood Market Retained 1985–2023': 18167885.47 acres


18167885.470937498

In [32]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_lost_1985_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **softwood** market (1985–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 7.12 acres

✅ Total forest area in LOST **softwood** market (1985–2023): 7.12 acres


# 1994 to 2023

In [21]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Softwood Sawlogs (1994–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_added_1994_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_retained_1994_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Softwood Market Added 1994–2023")
compute_forest_area(retained_shp, raster_path, "Softwood Market Retained 1994–2023")



🔍 Processing Softwood Market Added 1994–2023
Polygon 0: 9436.32 acres
✅ Total forest area in 'Softwood Market Added 1994–2023': 9436.32 acres

🔍 Processing Softwood Market Retained 1994–2023
Polygon 0: 18753632.43 acres
✅ Total forest area in 'Softwood Market Retained 1994–2023': 18753632.43 acres


18753632.429375

In [33]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_lost_1994_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **softwood** market (1994–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 37.58 acres

✅ Total forest area in LOST **softwood** market (1994–2023): 37.58 acres


# 2002 to 2023 

In [22]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Softwood Sawlogs (2002–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_added_2002_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_retained_2002_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Softwood Market Added 2002–2023")
compute_forest_area(retained_shp, raster_path, "Softwood Market Retained 2002–2023")



🔍 Processing Softwood Market Added 2002–2023
Polygon 0: 602.32 acres
✅ Total forest area in 'Softwood Market Added 2002–2023': 602.32 acres

🔍 Processing Softwood Market Retained 2002–2023
Polygon 0: 18762466.43 acres
✅ Total forest area in 'Softwood Market Retained 2002–2023': 18762466.43 acres


18762466.433125004

In [34]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_lost_2002_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **softwood** market (2002–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 2255.97 acres

✅ Total forest area in LOST **softwood** market (2002–2023): 2255.97 acres


# 2018 to 2023 

In [23]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Softwood Sawlogs (2018–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_added_2018_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_retained_2018_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Softwood Market Added 2018–2023")
compute_forest_area(retained_shp, raster_path, "Softwood Market Retained 2018–2023")



🔍 Processing Softwood Market Added 2018–2023
Polygon 0: 157220.56 acres
✅ Total forest area in 'Softwood Market Added 2018–2023': 157220.56 acres

🔍 Processing Softwood Market Retained 2018–2023
Polygon 0: 18605848.20 acres
✅ Total forest area in 'Softwood Market Retained 2018–2023': 18605848.20 acres


18605848.1953125

In [35]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\softwood_market_lost_2018_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **softwood** market (2018–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 11.34 acres

✅ Total forest area in LOST **softwood** market (2018–2023): 11.34 acres


# Pulpwood 

# 1985 to 2023

In [24]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Pulpwood (1985–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_added_1985_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_retained_1985_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Pulpwood Market Added 1985–2023")
compute_forest_area(retained_shp, raster_path, "Pulpwood Market Retained 1985–2023")



🔍 Processing Pulpwood Market Added 1985–2023
Polygon 0: 2940410.50 acres
✅ Total forest area in 'Pulpwood Market Added 1985–2023': 2940410.50 acres

🔍 Processing Pulpwood Market Retained 1985–2023
Polygon 0: 14743226.39 acres
✅ Total forest area in 'Pulpwood Market Retained 1985–2023': 14743226.39 acres


14743226.3878125

In [36]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_lost_1985_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **pulpwood** market (1985–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 1127616.84 acres

✅ Total forest area in LOST **pulpwood** market (1985–2023): 1127616.84 acres


# 1994 to 2023

In [25]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Pulpwood (1994–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_added_1994_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_retained_1994_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Pulpwood Market Added 1994–2023")
compute_forest_area(retained_shp, raster_path, "Pulpwood Market Retained 1994–2023")



🔍 Processing Pulpwood Market Added 1994–2023
Polygon 0: 8924274.30 acres
✅ Total forest area in 'Pulpwood Market Added 1994–2023': 8924274.30 acres

🔍 Processing Pulpwood Market Retained 1994–2023
Polygon 0: 8759362.60 acres
✅ Total forest area in 'Pulpwood Market Retained 1994–2023': 8759362.60 acres


8759362.595937502

In [37]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_lost_1994_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **pulpwood** market (1994–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 545623.56 acres

✅ Total forest area in LOST **pulpwood** market (1994–2023): 545623.56 acres


# 2002 to 2023

In [26]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Pulpwood (2002–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_added_2002_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_retained_2002_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Pulpwood Market Added 2002–2023")
compute_forest_area(retained_shp, raster_path, "Pulpwood Market Retained 2002–2023")



🔍 Processing Pulpwood Market Added 2002–2023
Polygon 0: 5376016.38 acres
✅ Total forest area in 'Pulpwood Market Added 2002–2023': 5376016.38 acres

🔍 Processing Pulpwood Market Retained 2002–2023
Polygon 0: 12307620.51 acres
✅ Total forest area in 'Pulpwood Market Retained 2002–2023': 12307620.51 acres


12307620.51125

In [38]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_lost_2002_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **pulpwood** market (2002–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 1204579.13 acres

✅ Total forest area in LOST **pulpwood** market (2002–2023): 1204579.13 acres


# 2018 to 2023

In [27]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# Updated shapefiles for Pulpwood (2018–2023)
added_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_added_2018_2023.shp"
retained_shp = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_retained_2018_2023.shp"

# === Forest group codes ===
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# === Function to compute forest area from shapefile ===
def compute_forest_area(shapefile_path, raster_path, label):
    with rasterio.open(raster_path) as src:
        pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
        nodata = src.nodata
        raster_crs = src.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        total_area = 0.0
        print(f"\n🔍 Processing {label}")

        with fiona.open(shapefile_path, 'r') as shp:
            shp_crs = shp.crs

            for feature in shp:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️ Polygon {polygon_id} skipped (no overlap)")
                        continue
                    else:
                        raise

                data = out_image[0]
                valid_data = data[data != nodata] if nodata is not None else data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)
                forest_type_areas = {
                    forest_type_mapping.get(ft, None): count * pixel_area
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas = {
                    k: v for k, v in forest_type_areas.items() if k is not None
                }

                area = sum(forest_type_areas.values())
                total_area += area

                print(f"Polygon {polygon_id}: {area:.2f} acres")

        print(f"✅ Total forest area in '{label}': {total_area:.2f} acres")
        return total_area

# === Run both ===
compute_forest_area(added_shp, raster_path, "Pulpwood Market Added 2018–2023")
compute_forest_area(retained_shp, raster_path, "Pulpwood Market Retained 2018–2023")



🔍 Processing Pulpwood Market Added 2018–2023
Polygon 0: 3460134.09 acres
✅ Total forest area in 'Pulpwood Market Added 2018–2023': 3460134.09 acres

🔍 Processing Pulpwood Market Retained 2018–2023
Polygon 0: 14223502.80 acres
✅ Total forest area in 'Pulpwood Market Retained 2018–2023': 14223502.80 acres


14223502.7965625

In [39]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# === Correct paths ===
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\lulc2001.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\pulpwood_market_lost_2018_2023.shp"

# === Forest class values (NLCD) ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Read raster ===
with rasterio.open(raster_path) as src:
    pixel_area_acres = (src.res[0] * abs(src.res[1])) * 0.000247105  # Convert m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    # === Function to reproject geometries if needed ===
    def reproject_geom(geom, src_crs, dst_crs):
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(transformer, shape(geom)))

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs
        total_forest_area = 0.0

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, _ = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"⚠️ Polygon {polygon_id} does not overlap raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]
            forest_mask = np.isin(data, forest_values)
            if nodata is not None:
                forest_mask &= (data != nodata)

            forest_area_acres = np.sum(forest_mask) * pixel_area_acres
            total_forest_area += forest_area_acres

            print(f"🌲 Polygon {polygon_id} forest area: {forest_area_acres:.2f} acres")

        print(f"\n✅ Total forest area in LOST **pulpwood** market (2018–2023): {total_forest_area:.2f} acres")


🌲 Polygon 0 forest area: 222280.41 acres

✅ Total forest area in LOST **pulpwood** market (2018–2023): 222280.41 acres


**Major Calulations of the AREA**

**2018 Hardwood**

In [47]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np

# Optional: for reprojecting geometries if needed
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Path to the clipped raster
clipped_raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\Forest products journal\GIS\Randoms\clipped_forestgroup.tif"

# Path to the shapefile containing the 5 polygons
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2018\hard\hard18.shp"

# Forest type mapping (from the attribute table)
forest_type_mapping = {
    100: "White/Red/Jack Pine Group",
    120: "Spruce/Fir Group",
    140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group",
    180: "Pinyon/Juniper Group",
    200: "Douglas-fir Group",
    220: "Ponderosa Pine Group",
    240: "Western White Pine Group",
    260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group",
    300: "Hemlock/Sitka Spruce Group",
    320: "Western Larch Group",
    340: "Redwood Group",
    360: "Other Western Softwood Group",
    370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group",
    400: "Oak/Pine Group",
    500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group",
    700: "Elm/Ash/Cottonwood Group",
    800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group",
    910: "Alder/Maple Group",
    920: "Western Oak Group",
    940: "Tanoak/Laurel Group",
    950: "Other Western Hardwoods Group",
    980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# Open the raster file
with rasterio.open(clipped_raster_path) as src:
    # Get pixel size and compute area of one pixel in square meters
    pixel_width = src.transform[0]  # pixel width in CRS units
    pixel_height = abs(src.transform[4])  # pixel height in CRS units
    pixel_area_sq_m = pixel_width * pixel_height
    # Convert pixel area to acres (1 m² = 0.000247105 acres)
    pixel_area_acres = pixel_area_sq_m * 0.000247105

    # Read the raster's nodata value (if defined)
    nodata = src.nodata

    # Store raster CRS for later comparison
    raster_crs = src.crs

    # Open the shapefile containing the polygons
    with fiona.open(shapefile_path, 'r') as shapefile:
        shp_crs = shapefile.crs  # shapefile CRS
        
        # Define a function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(project, shape(geom)))
        
        # Iterate over each polygon feature in the shapefile
        for feature in shapefile:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry if CRS differ
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # We assume the raster has one band; select the first band.
            out_data = out_image[0]

            # Exclude pixels with nodata value (if defined)
            if nodata is not None:
                valid_data = out_data[out_data != nodata]
            else:
                valid_data = out_data

            # Calculate unique forest types and their pixel counts in the masked area.
            unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

            # Map forest type codes to names and compute the area (in acres)
            forest_type_areas_acres = {
                forest_type_mapping.get(forest_type, None): count * pixel_area_acres
                for forest_type, count in zip(unique_types, pixel_counts)
            }
            # Remove any entries with unknown forest types (i.e. keys that are None)
            forest_type_areas_acres = {
                name: area for name, area in forest_type_areas_acres.items() if name is not None
            }

            # Calculate the total forest area within the polygon
            total_area_acres = sum(forest_type_areas_acres.values())

            # Display the results for the current polygon
            print(f"Results for Polygon {polygon_id}:")
            for forest_type, area in forest_type_areas_acres.items():
                print(f"   {forest_type}: {area:.2f} acres")
            print(f"   Total Forest Area: {total_area_acres:.2f} acres\n")


Results for Polygon 0:
   White/Red/Jack Pine Group: 316433.40 acres
   Spruce/Fir Group: 1494475.60 acres
   Exotic Softwoods Group: 139.00 acres
   Oak/Hickory Group: 9745.20 acres
   Elm/Ash/Cottonwood Group: 46532.96 acres
   Maple/Beech/Birch Group: 3812258.72 acres
   Aspen/Birch Group: 938999.00 acres
   Total Forest Area: 6618583.87 acres

Results for Polygon 1:
   White/Red/Jack Pine Group: 242935.10 acres
   Spruce/Fir Group: 564264.27 acres
   Oak/Hickory Group: 4355.23 acres
   Elm/Ash/Cottonwood Group: 5111.98 acres
   Maple/Beech/Birch Group: 117822.75 acres
   Aspen/Birch Group: 403167.25 acres
   Total Forest Area: 1337656.59 acres

Results for Polygon 2:
   White/Red/Jack Pine Group: 116973.33 acres
   Spruce/Fir Group: 444665.45 acres
   Oak/Hickory Group: 30.89 acres
   Maple/Beech/Birch Group: 77.22 acres
   Aspen/Birch Group: 469159.73 acres
   Total Forest Area: 1030906.62 acres

Results for Polygon 3:
   White/Red/Jack Pine Group: 197853.88 acres
   Spruce/Fir Gr

Softwood 2018

In [48]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np

# Optional: for reprojecting geometries if needed
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Path to the clipped raster
clipped_raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\Forest products journal\GIS\Randoms\clipped_forestgroup.tif"

# Path to the shapefile containing the 5 polygons
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2018\soft\softwood18.shp"

# Forest type mapping (from the attribute table)
forest_type_mapping = {
    100: "White/Red/Jack Pine Group",
    120: "Spruce/Fir Group",
    140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group",
    180: "Pinyon/Juniper Group",
    200: "Douglas-fir Group",
    220: "Ponderosa Pine Group",
    240: "Western White Pine Group",
    260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group",
    300: "Hemlock/Sitka Spruce Group",
    320: "Western Larch Group",
    340: "Redwood Group",
    360: "Other Western Softwood Group",
    370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group",
    400: "Oak/Pine Group",
    500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group",
    700: "Elm/Ash/Cottonwood Group",
    800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group",
    910: "Alder/Maple Group",
    920: "Western Oak Group",
    940: "Tanoak/Laurel Group",
    950: "Other Western Hardwoods Group",
    980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# Open the raster file
with rasterio.open(clipped_raster_path) as src:
    # Get pixel size and compute area of one pixel in square meters
    pixel_width = src.transform[0]  # pixel width in CRS units
    pixel_height = abs(src.transform[4])  # pixel height in CRS units
    pixel_area_sq_m = pixel_width * pixel_height
    # Convert pixel area to acres (1 m² = 0.000247105 acres)
    pixel_area_acres = pixel_area_sq_m * 0.000247105

    # Read the raster's nodata value (if defined)
    nodata = src.nodata

    # Store raster CRS for later comparison
    raster_crs = src.crs

    # Open the shapefile containing the polygons
    with fiona.open(shapefile_path, 'r') as shapefile:
        shp_crs = shapefile.crs  # shapefile CRS
        
        # Define a function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(project, shape(geom)))
        
        # Iterate over each polygon feature in the shapefile
        for feature in shapefile:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry if CRS differ
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # We assume the raster has one band; select the first band.
            out_data = out_image[0]

            # Exclude pixels with nodata value (if defined)
            if nodata is not None:
                valid_data = out_data[out_data != nodata]
            else:
                valid_data = out_data

            # Calculate unique forest types and their pixel counts in the masked area.
            unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

            # Map forest type codes to names and compute the area (in acres)
            forest_type_areas_acres = {
                forest_type_mapping.get(forest_type, None): count * pixel_area_acres
                for forest_type, count in zip(unique_types, pixel_counts)
            }
            # Remove any entries with unknown forest types (i.e. keys that are None)
            forest_type_areas_acres = {
                name: area for name, area in forest_type_areas_acres.items() if name is not None
            }

            # Calculate the total forest area within the polygon
            total_area_acres = sum(forest_type_areas_acres.values())

            # Display the results for the current polygon
            print(f"Results for Polygon {polygon_id}:")
            for forest_type, area in forest_type_areas_acres.items():
                print(f"   {forest_type}: {area:.2f} acres")
            print(f"   Total Forest Area: {total_area_acres:.2f} acres\n")


Results for Polygon 0:
   White/Red/Jack Pine Group: 669036.79 acres
   Spruce/Fir Group: 1999990.65 acres
   Exotic Softwoods Group: 139.00 acres
   Oak/Pine Group: 30.89 acres
   Oak/Hickory Group: 149235.98 acres
   Elm/Ash/Cottonwood Group: 151907.80 acres
   Maple/Beech/Birch Group: 3451608.97 acres
   Aspen/Birch Group: 2889460.54 acres
   Total Forest Area: 9311410.61 acres

Results for Polygon 1:
   White/Red/Jack Pine Group: 463769.75 acres
   Spruce/Fir Group: 801546.84 acres
   Oak/Hickory Group: 3598.47 acres
   Elm/Ash/Cottonwood Group: 24386.17 acres
   Maple/Beech/Birch Group: 349591.80 acres
   Aspen/Birch Group: 2478864.70 acres
   Total Forest Area: 4121757.73 acres

Results for Polygon 2:
   White/Red/Jack Pine Group: 589777.86 acres
   Spruce/Fir Group: 127027.41 acres
   Oak/Hickory Group: 540.54 acres
   Elm/Ash/Cottonwood Group: 401.55 acres
   Aspen/Birch Group: 1656869.91 acres
   Total Forest Area: 2374617.27 acres

Results for Polygon 3:
   White/Red/Jack Pin

Biomass 2018

In [49]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np

# Optional: for reprojecting geometries if needed
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Path to the clipped raster
clipped_raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\Forest products journal\GIS\Randoms\clipped_forestgroup.tif"

# Path to the shapefile containing the 5 polygons
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2018\bio\biomass18.shp"

# Forest type mapping (from the attribute table)
forest_type_mapping = {
    100: "White/Red/Jack Pine Group",
    120: "Spruce/Fir Group",
    140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group",
    180: "Pinyon/Juniper Group",
    200: "Douglas-fir Group",
    220: "Ponderosa Pine Group",
    240: "Western White Pine Group",
    260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group",
    300: "Hemlock/Sitka Spruce Group",
    320: "Western Larch Group",
    340: "Redwood Group",
    360: "Other Western Softwood Group",
    370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group",
    400: "Oak/Pine Group",
    500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group",
    700: "Elm/Ash/Cottonwood Group",
    800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group",
    910: "Alder/Maple Group",
    920: "Western Oak Group",
    940: "Tanoak/Laurel Group",
    950: "Other Western Hardwoods Group",
    980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# Open the raster file
with rasterio.open(clipped_raster_path) as src:
    # Get pixel size and compute area of one pixel in square meters
    pixel_width = src.transform[0]  # pixel width in CRS units
    pixel_height = abs(src.transform[4])  # pixel height in CRS units
    pixel_area_sq_m = pixel_width * pixel_height
    # Convert pixel area to acres (1 m² = 0.000247105 acres)
    pixel_area_acres = pixel_area_sq_m * 0.000247105

    # Read the raster's nodata value (if defined)
    nodata = src.nodata

    # Store raster CRS for later comparison
    raster_crs = src.crs

    # Open the shapefile containing the polygons
    with fiona.open(shapefile_path, 'r') as shapefile:
        shp_crs = shapefile.crs  # shapefile CRS
        
        # Define a function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(project, shape(geom)))
        
        # Iterate over each polygon feature in the shapefile
        for feature in shapefile:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry if CRS differ
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # We assume the raster has one band; select the first band.
            out_data = out_image[0]

            # Exclude pixels with nodata value (if defined)
            if nodata is not None:
                valid_data = out_data[out_data != nodata]
            else:
                valid_data = out_data

            # Calculate unique forest types and their pixel counts in the masked area.
            unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

            # Map forest type codes to names and compute the area (in acres)
            forest_type_areas_acres = {
                forest_type_mapping.get(forest_type, None): count * pixel_area_acres
                for forest_type, count in zip(unique_types, pixel_counts)
            }
            # Remove any entries with unknown forest types (i.e. keys that are None)
            forest_type_areas_acres = {
                name: area for name, area in forest_type_areas_acres.items() if name is not None
            }

            # Calculate the total forest area within the polygon
            total_area_acres = sum(forest_type_areas_acres.values())

            # Display the results for the current polygon
            print(f"Results for Polygon {polygon_id}:")
            for forest_type, area in forest_type_areas_acres.items():
                print(f"   {forest_type}: {area:.2f} acres")
            print(f"   Total Forest Area: {total_area_acres:.2f} acres\n")


Results for Polygon 0:
   White/Red/Jack Pine Group: 353684.48 acres
   Spruce/Fir Group: 1156621.28 acres
   Exotic Softwoods Group: 139.00 acres
   Oak/Hickory Group: 68587.08 acres
   Elm/Ash/Cottonwood Group: 94672.10 acres
   Maple/Beech/Birch Group: 3416859.83 acres
   Aspen/Birch Group: 2065983.13 acres
   Total Forest Area: 7156546.90 acres

Results for Polygon 1:
   White/Red/Jack Pine Group: 732264.78 acres
   Spruce/Fir Group: 349730.80 acres
   Oak/Hickory Group: 6718.17 acres
   Elm/Ash/Cottonwood Group: 5791.52 acres
   Maple/Beech/Birch Group: 8772.23 acres
   Aspen/Birch Group: 2783545.16 acres
   Total Forest Area: 3886822.65 acres

Results for Polygon 2:
   White/Red/Jack Pine Group: 589252.76 acres
   Spruce/Fir Group: 286394.70 acres
   Oak/Hickory Group: 247.11 acres
   Elm/Ash/Cottonwood Group: 1127.42 acres
   Maple/Beech/Birch Group: 61.78 acres
   Aspen/Birch Group: 2106292.13 acres
   Total Forest Area: 2983375.89 acres

Results for Polygon 3:
   White/Red/Jac

Pulp 2018

In [50]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np

# Optional: for reprojecting geometries if needed
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Path to the clipped raster
clipped_raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\Forest products journal\GIS\Randoms\clipped_forestgroup.tif"

# Path to the shapefile containing the 5 polygons
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2018\pulp\pulp18.shp"

# Forest type mapping (from the attribute table)
forest_type_mapping = {
    100: "White/Red/Jack Pine Group",
    120: "Spruce/Fir Group",
    140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group",
    180: "Pinyon/Juniper Group",
    200: "Douglas-fir Group",
    220: "Ponderosa Pine Group",
    240: "Western White Pine Group",
    260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group",
    300: "Hemlock/Sitka Spruce Group",
    320: "Western Larch Group",
    340: "Redwood Group",
    360: "Other Western Softwood Group",
    370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group",
    400: "Oak/Pine Group",
    500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group",
    700: "Elm/Ash/Cottonwood Group",
    800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group",
    910: "Alder/Maple Group",
    920: "Western Oak Group",
    940: "Tanoak/Laurel Group",
    950: "Other Western Hardwoods Group",
    980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# Open the raster file
with rasterio.open(clipped_raster_path) as src:
    # Get pixel size and compute area of one pixel in square meters
    pixel_width = src.transform[0]  # pixel width in CRS units
    pixel_height = abs(src.transform[4])  # pixel height in CRS units
    pixel_area_sq_m = pixel_width * pixel_height
    # Convert pixel area to acres (1 m² = 0.000247105 acres)
    pixel_area_acres = pixel_area_sq_m * 0.000247105

    # Read the raster's nodata value (if defined)
    nodata = src.nodata

    # Store raster CRS for later comparison
    raster_crs = src.crs

    # Open the shapefile containing the polygons
    with fiona.open(shapefile_path, 'r') as shapefile:
        shp_crs = shapefile.crs  # shapefile CRS
        
        # Define a function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(project, shape(geom)))
        
        # Iterate over each polygon feature in the shapefile
        for feature in shapefile:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry if CRS differ
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # We assume the raster has one band; select the first band.
            out_data = out_image[0]

            # Exclude pixels with nodata value (if defined)
            if nodata is not None:
                valid_data = out_data[out_data != nodata]
            else:
                valid_data = out_data

            # Calculate unique forest types and their pixel counts in the masked area.
            unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

            # Map forest type codes to names and compute the area (in acres)
            forest_type_areas_acres = {
                forest_type_mapping.get(forest_type, None): count * pixel_area_acres
                for forest_type, count in zip(unique_types, pixel_counts)
            }
            # Remove any entries with unknown forest types (i.e. keys that are None)
            forest_type_areas_acres = {
                name: area for name, area in forest_type_areas_acres.items() if name is not None
            }

            # Calculate the total forest area within the polygon
            total_area_acres = sum(forest_type_areas_acres.values())

            # Display the results for the current polygon
            print(f"Results for Polygon {polygon_id}:")
            for forest_type, area in forest_type_areas_acres.items():
                print(f"   {forest_type}: {area:.2f} acres")
            print(f"   Total Forest Area: {total_area_acres:.2f} acres\n")


Results for Polygon 0:
   White/Red/Jack Pine Group: 1641394.96 acres
   Spruce/Fir Group: 896095.39 acres
   Exotic Softwoods Group: 123.55 acres
   Oak/Pine Group: 30.89 acres
   Oak/Hickory Group: 38146.83 acres
   Elm/Ash/Cottonwood Group: 59058.10 acres
   Maple/Beech/Birch Group: 1453039.18 acres
   Aspen/Birch Group: 5300587.58 acres
   Total Forest Area: 9388476.48 acres

Results for Polygon 1:
   White/Red/Jack Pine Group: 290518.26 acres
   Spruce/Fir Group: 1603742.34 acres
   Exotic Softwoods Group: 15.44 acres
   Oak/Hickory Group: 11922.82 acres
   Elm/Ash/Cottonwood Group: 40911.32 acres
   Maple/Beech/Birch Group: 2203018.30 acres
   Aspen/Birch Group: 860373.28 acres
   Total Forest Area: 5010501.75 acres



**2002**

**hardwood 2002**

In [51]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2002\hard\hardwood02.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 105959.19 acres.
Polygon 1 has a forest area of 3921757.54 acres.
Polygon 2 has a forest area of 471267.96 acres.
Polygon 3 has a forest area of 993472.93 acres.
Polygon 4 has a forest area of 10109689.02 acres.


**Softwood 2002**

In [52]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2002\soft\softwood02.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 7890.33 acres.
Polygon 1 has a forest area of 657284.94 acres.
Polygon 2 has a forest area of 345770.07 acres.
Polygon 3 has a forest area of 557896.84 acres.
Polygon 4 has a forest area of 14027332.04 acres.


Biomass 2002

In [53]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2002\bio\biomass02.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 1135885.25 acres.
Polygon 1 has a forest area of 5248847.79 acres.


**Pulp 2002**

In [54]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\2002\pulp\pulpwood02.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 3551698.65 acres.
Polygon 1 has a forest area of 7818600.01 acres.


**1994**

**Hardwood 1994**

In [55]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\1994\hard\hardwood94.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 12267.28 acres.
Polygon 1 has a forest area of 3562274.85 acres.
Polygon 2 has a forest area of 671294.24 acres.
Polygon 3 has a forest area of 619647.79 acres.
Polygon 4 has a forest area of 10737275.62 acres.


**Softwood 1994**

In [56]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\1994\soft\softwood94.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 1127940.20 acres.
Polygon 1 has a forest area of 3101094.26 acres.
Polygon 2 has a forest area of 1827605.75 acres.
Polygon 3 has a forest area of 2858696.27 acres.
Polygon 4 has a forest area of 6671008.36 acres.


In [3]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj
import os

# === File paths ===
base_dir = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps"

raster_path = os.path.join(base_dir, "Market_Change_SHP", "lulc20011.tif")
shapefile_path = os.path.join(base_dir, "softwood_1994.shp")

# === Forest values from NLCD or similar LULC classification ===
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# === Open raster ===
with rasterio.open(raster_path) as src:
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # m² to acres
    nodata = src.nodata
    raster_crs = src.crs

    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs

        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        for feature in shp:
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            data = out_image[0]

            if nodata is not None:
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            forest_area_acres = np.sum(forest_mask) * pixel_area
            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 15587859.57 acres.


**Biomass 1994**

In [57]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\1994\bio\biomass94.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 1651467.98 acres.
Polygon 1 has a forest area of 5117883.45 acres.


**Pulpwood 1994**

Pulp hotspot 

In [63]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\Forest products journal\GIS\Mills\1994_location\Pulp_1994\Competition 1\hotspotdisolve.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 10018044.25 acres.


**Pulp 1994 coldspot**

In [64]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\Forest products journal\GIS\Mills\1994_location\Pulp_1994\Competition 1\coldspotdisolve.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 4165778.13 acres.


**1985**

**Hardwood 1985**

In [59]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\1985\hard\hardwood85.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 9852058.56 acres.
Polygon 1 has a forest area of 5520890.76 acres.
Polygon 2 has a forest area of 189849.51 acres.


**Softwood 1985**

In [60]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\1985\soft\softwood85.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 761585.52 acres.
Polygon 1 has a forest area of 10216471.30 acres.
Polygon 2 has a forest area of 3653432.80 acres.
Polygon 3 has a forest area of 531582.23 acres.
Polygon 4 has a forest area of 3997.99 acres.


**Biomass 1985**

In [61]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\1985\bio\biomass85.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 3103480.11 acres.


**Pulpwood 1985**

In [62]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# Define the raster and shapefile paths
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"
shapefile_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\Shp files for different classes\1985\pulp\pulpwood85.shp"

# Define the forest values to be included (for example, values: 31, 41, 42, 43, 52, 71, 81, 95)
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Open the raster file
with rasterio.open(raster_path) as src:
    # Calculate the area of one pixel in acres.
    # Note: src.res returns the (pixel_width, pixel_height) in the CRS units (usually meters)
    pixel_area = (src.res[0] * src.res[1]) * 0.000247105  # 1 m² = 0.000247105 acres
    nodata = src.nodata
    raster_crs = src.crs  # CRS of the raster

    # Open the shapefile with polygons
    with fiona.open(shapefile_path, 'r') as shp:
        shp_crs = shp.crs  # CRS of the shapefile

        # Function to reproject geometries if needed
        def reproject_geom(geom, src_crs, dst_crs):
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
            return mapping(transform(transformer, shape(geom)))

        # Iterate over each polygon feature in the shapefile
        for feature in shp:
            # Use an attribute or the feature ID as an identifier for the polygon.
            polygon_id = feature.get("id", "unknown")
            geom = feature["geometry"]

            # Reproject geometry to raster CRS if necessary
            if shp_crs != raster_crs:
                geom = reproject_geom(geom, shp_crs, raster_crs)

            try:
                # Clip the raster by the polygon geometry.
                # The resulting masked array has pixels outside the polygon masked.
                out_image, out_transform = mask(src, [geom], crop=True)
            except ValueError as e:
                # Skip polygons that do not overlap the raster.
                if "do not overlap" in str(e):
                    print(f"Polygon {polygon_id} does not overlap the raster. Skipping...")
                    continue
                else:
                    raise

            # Assume the raster has one band; select that band.
            data = out_image[0]

            # Create a mask for forest values.
            # If a nodata value is defined, exclude those pixels.
            if nodata is not None:
                valid_data = data[data != nodata]
                # When computing the forest mask, we'll compare only valid pixels.
                # Create a boolean mask the same shape as 'data'
                forest_mask = np.isin(data, forest_values) & (data != nodata)
            else:
                forest_mask = np.isin(data, forest_values)

            # Compute the forest area (in acres) by counting pixels that match forest values.
            forest_area_acres = np.sum(forest_mask) * pixel_area

            print(f"Polygon {polygon_id} has a forest area of {forest_area_acres:.2f} acres.")


Polygon 0 has a forest area of 7074032.56 acres.
Polygon 1 has a forest area of 5868673.05 acres.


# 2018 Forest cover area 

In [24]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np

# Optional: for reprojecting geometries if needed
from shapely.geometry import mapping

# --- File paths ---
clipped_raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\Forest products journal\GIS\Randoms\clipped_forestgroup.tif"
michigan_fp = r"\\for-stor.forestry.oregonstate.edu\home\I-K\khanalna\Documents\ArcGIS\Packages\HW3_0021d6\commondata\michigan_shp\Counties_(v17a).shp"

# --- Forest type mapping ---
forest_type_mapping = {
    100: "White/Red/Jack Pine Group",
    120: "Spruce/Fir Group",
    140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group",
    180: "Pinyon/Juniper Group",
    200: "Douglas-fir Group",
    220: "Ponderosa Pine Group",
    240: "Western White Pine Group",
    260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group",
    300: "Hemlock/Sitka Spruce Group",
    320: "Western Larch Group",
    340: "Redwood Group",
    360: "Other Western Softwood Group",
    370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group",
    400: "Oak/Pine Group",
    500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group",
    700: "Elm/Ash/Cottonwood Group",
    800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group",
    910: "Alder/Maple Group",
    920: "Western Oak Group",
    940: "Tanoak/Laurel Group",
    950: "Other Western Hardwoods Group",
    980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# Load Michigan counties shapefile and dissolve into one geometry
gdf_michigan = gpd.read_file(michigan_fp)
michigan_geom = gdf_michigan.unary_union

# Open raster and process
with rasterio.open(clipped_raster_path) as src:
    # Get CRS and reproject Michigan shape if needed
    raster_crs = src.crs
    gdf_michigan = gdf_michigan.to_crs(raster_crs)
    michigan_geom_proj = gdf_michigan.unary_union

    # Compute pixel area in acres
    pixel_width = src.transform[0]
    pixel_height = abs(src.transform[4])
    pixel_area_acres = pixel_width * pixel_height * 0.000247105

    # Mask the raster using Michigan boundary
    out_image, out_transform = mask(src, [mapping(michigan_geom_proj)], crop=True)
    out_data = out_image[0]

    # Remove nodata
    nodata = src.nodata
    if nodata is not None:
        valid_data = out_data[out_data != nodata]
    else:
        valid_data = out_data

    # Compute pixel count by forest type code
    unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

    # Convert codes to names and compute area
    forest_type_areas = {
        forest_type_mapping.get(code, None): count * pixel_area_acres
        for code, count in zip(unique_types, pixel_counts)
    }

    # Remove unknown types
    forest_type_areas = {
        k: v for k, v in forest_type_areas.items() if k is not None
    }

    # Print results
    print(f"🌲 Forest cover in Michigan by forest type:")
    total_acres = 0
    for forest_type, area in forest_type_areas.items():
        print(f"   {forest_type}: {area:,.2f} acres")
        total_acres += area
    print(f"\n🟩 Total forested area in Michigan: {total_acres:,.2f} acres")


🌲 Forest cover in Michigan by forest type:
   White/Red/Jack Pine Group: 2,435,791.21 acres
   Spruce/Fir Group: 3,250,141.18 acres
   Exotic Softwoods Group: 139.00 acres
   Oak/Pine Group: 30.89 acres
   Oak/Hickory Group: 154,317.07 acres
   Elm/Ash/Cottonwood Group: 179,799.78 acres
   Maple/Beech/Birch Group: 4,085,927.51 acres
   Aspen/Birch Group: 8,882,003.90 acres

🟩 Total forested area in Michigan: 18,988,150.52 acres


# 2001 Forest cover 

In [25]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
from shapely.geometry import mapping

# File paths
michigan_fp = r"\\for-stor.forestry.oregonstate.edu\home\I-K\khanalna\Documents\ArcGIS\Packages\HW3_0021d6\commondata\michigan_shp\Counties_(v17a).shp"
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\Jan 2025\MSC thesis II\GIS\GIS After Meeting June 5\2001 LULC\lulc20011.tif"

# Define land cover values considered as forest or woody cover
forest_values = [31, 41, 42, 43, 52, 71, 81, 95]

# Load Michigan counties and dissolve into a single polygon
gdf_michigan = gpd.read_file(michigan_fp)
gdf_michigan = gdf_michigan.to_crs("EPSG:5070")  # Optional reproject
michigan_geom = gdf_michigan.unary_union

# Open and process raster
with rasterio.open(raster_path) as src:
    gdf_michigan = gdf_michigan.to_crs(src.crs)
    michigan_geom_proj = gdf_michigan.unary_union

    # Calculate pixel area in acres
    pixel_area_acres = src.res[0] * abs(src.res[1]) * 0.000247105

    # Clip raster with Michigan geometry
    out_image, out_transform = mask(src, [mapping(michigan_geom_proj)], crop=True)
    data = out_image[0]

    # Exclude nodata
    nodata = src.nodata
    if nodata is not None:
        data = data[data != nodata]

    # Count pixels matching forest values
    forest_mask = np.isin(data, forest_values)
    forest_pixel_count = np.sum(forest_mask)
    forest_area_acres = forest_pixel_count * pixel_area_acres
    forest_area_ha = forest_area_acres * 0.404686

    # Output
    print("🌲 Michigan Forest Cover in 2001 (based on selected LULC classes):")
    print(f"   - {forest_area_acres:,.2f} acres")
    print(f"   - {forest_area_ha:,.2f} hectares")


🌲 Michigan Forest Cover in 2001 (based on selected LULC classes):
   - 15,819,317.31 acres
   - 6,401,856.25 hectares


# Analysis for 2023

# Biomass 2023 Forest cover area 

In [40]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# --- Updated raster path ---
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# --- Updated dictionary of competition level shapefiles ---
shapefiles = {
    "lowest": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\biomass_2023_lowest.shp",
    "low":    r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\biomass_2023_low.shp",
    "medium": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\biomass_2023_medium.shp"
}

# --- Forest type mapping ---
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# --- Open raster once ---
with rasterio.open(raster_path) as src:
    pixel_width = src.transform[0]
    pixel_height = abs(src.transform[4])
    pixel_area_sq_m = pixel_width * pixel_height
    pixel_area_acres = pixel_area_sq_m * 0.000247105
    nodata = src.nodata
    raster_crs = src.crs

    # --- Function to reproject geometries ---
    def reproject_geom(geom, src_crs, dst_crs):
        project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(project, shape(geom)))

    # --- Loop through each shapefile ---
    for level, shapefile_path in shapefiles.items():
        print(f"\n🔍 Processing '{level}' competition zone...")

        with fiona.open(shapefile_path, 'r') as shapefile:
            shp_crs = shapefile.crs

            total_zone_area = 0.0  # Track total area per shapefile

            for feature in shapefile:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️  Polygon {polygon_id} does not overlap the raster. Skipping...")
                        continue
                    else:
                        raise

                out_data = out_image[0]

                # Remove nodata pixels
                valid_data = out_data[out_data != nodata] if nodata is not None else out_data

                # Forest type counts
                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

                forest_type_areas_acres = {
                    forest_type_mapping.get(ft, None): count * pixel_area_acres
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas_acres = {
                    name: area for name, area in forest_type_areas_acres.items() if name is not None
                }

                polygon_area = sum(forest_type_areas_acres.values())
                total_zone_area += polygon_area

            print(f"✅ Total forest area in '{level}' zone: {total_zone_area:.2f} acres")



🔍 Processing 'lowest' competition zone...
✅ Total forest area in 'lowest' zone: 8073445.45 acres

🔍 Processing 'low' competition zone...
✅ Total forest area in 'low' zone: 7019341.85 acres

🔍 Processing 'medium' competition zone...
✅ Total forest area in 'medium' zone: 25513.59 acres


# Hardwood 2023 Forest cover 

In [42]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# --- Raster path ---
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# --- Corrected shapefile paths (hardwood_[rank]_2023.shp) ---
shapefiles = {
    "highest": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\hardwood_2023_highest.shp",
    "high":    r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\hardwood_2023_high.shp",
    "medium":  r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\hardwood_2023_medium.shp",
    "low":     r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\hardwood_2023_low.shp",
    "lowest":  r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\hardwood_2023_lowest.shp"
}

# --- Forest type mapping ---
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# --- Open raster once ---
with rasterio.open(raster_path) as src:
    pixel_width = src.transform[0]
    pixel_height = abs(src.transform[4])
    pixel_area_sq_m = pixel_width * pixel_height
    pixel_area_acres = pixel_area_sq_m * 0.000247105
    nodata = src.nodata
    raster_crs = src.crs

    # --- Function to reproject geometries ---
    def reproject_geom(geom, src_crs, dst_crs):
        project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(project, shape(geom)))

    # --- Loop through shapefiles ---
    for level, shapefile_path in shapefiles.items():
        print(f"\n🔍 Processing '{level}' competition zone...")

        with fiona.open(shapefile_path, 'r') as shapefile:
            shp_crs = shapefile.crs
            total_zone_area = 0.0

            for feature in shapefile:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️  Polygon {polygon_id} does not overlap raster. Skipping...")
                        continue
                    else:
                        raise

                out_data = out_image[0]
                valid_data = out_data[out_data != nodata] if nodata is not None else out_data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

                forest_type_areas_acres = {
                    forest_type_mapping.get(ft, None): count * pixel_area_acres
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas_acres = {
                    name: area for name, area in forest_type_areas_acres.items() if name is not None
                }

                total_zone_area += sum(forest_type_areas_acres.values())

            print(f"✅ Total forest area in '{level}' zone: {total_zone_area:.2f} acres")



🔍 Processing 'highest' competition zone...
✅ Total forest area in 'highest' zone: 8991409.63 acres

🔍 Processing 'high' competition zone...
✅ Total forest area in 'high' zone: 1642645.93 acres

🔍 Processing 'medium' competition zone...
✅ Total forest area in 'medium' zone: 756913.50 acres

🔍 Processing 'low' competition zone...
✅ Total forest area in 'low' zone: 1082690.56 acres

🔍 Processing 'lowest' competition zone...
✅ Total forest area in 'lowest' zone: 6297100.27 acres


# Softwood 2023 

In [45]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# --- Path to the clipped forest group raster ---
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# --- Corrected dictionary of softwood competition shapefiles ---
shapefiles = {
    "highest": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\softwood_2023_competition_highest.shp",
    "high":    r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\softwood_2023_competition_high.shp",
    "medium":  r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\softwood_2023_competition_medium.shp",
    "low":     r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\softwood_2023_competition_low.shp",
    "lowest":  r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\softwood_2023_competition_lowest.shp"
}

# --- Forest type mapping ---
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# --- Open raster once ---
with rasterio.open(raster_path) as src:
    pixel_width = src.transform[0]
    pixel_height = abs(src.transform[4])
    pixel_area_sq_m = pixel_width * pixel_height
    pixel_area_acres = pixel_area_sq_m * 0.000247105
    nodata = src.nodata
    raster_crs = src.crs

    # --- Function to reproject geometries ---
    def reproject_geom(geom, src_crs, dst_crs):
        project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(project, shape(geom)))

    # --- Loop through each shapefile ---
    for level, shapefile_path in shapefiles.items():
        print(f"\n🔍 Processing '{level}' competition zone...")

        with fiona.open(shapefile_path, 'r') as shapefile:
            shp_crs = shapefile.crs
            total_zone_area = 0.0

            for feature in shapefile:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️  Polygon {polygon_id} does not overlap the raster. Skipping...")
                        continue
                    else:
                        raise

                out_data = out_image[0]
                valid_data = out_data[out_data != nodata] if nodata is not None else out_data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

                forest_type_areas_acres = {
                    forest_type_mapping.get(ft, None): count * pixel_area_acres
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas_acres = {
                    name: area for name, area in forest_type_areas_acres.items() if name is not None
                }

                total_zone_area += sum(forest_type_areas_acres.values())

            print(f"✅ Total forest area in '{level}' zone: {total_zone_area:.2f} acres")



🔍 Processing 'highest' competition zone...
✅ Total forest area in 'highest' zone: 10104308.78 acres

🔍 Processing 'high' competition zone...
✅ Total forest area in 'high' zone: 633947.88 acres

🔍 Processing 'medium' competition zone...
✅ Total forest area in 'medium' zone: 3576335.22 acres

🔍 Processing 'low' competition zone...
✅ Total forest area in 'low' zone: 2791128.20 acres

🔍 Processing 'lowest' competition zone...
✅ Total forest area in 'lowest' zone: 1657256.01 acres


# Pulpwood 2023

In [46]:
import rasterio
from rasterio.mask import mask
import fiona
import numpy as np
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# --- Raster path ---
raster_path = r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\MI Market Coverage Maps\Market_Change_SHP\forest_cover_2019.tif"

# --- Shapefile paths for pulpwood competition (low and lowest) ---
shapefiles = {
    "low": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\pulpwood_2023_competition_low.shp",
    "lowest": r"C:\Users\khanalna\OneDrive - Oregon State University\Naresh Personal\MSU Manuscript\June 25\Competition Hotspots Data\pulpwood_2023_competition_lowest.shp"
}

# --- Forest type mapping ---
forest_type_mapping = {
    100: "White/Red/Jack Pine Group", 120: "Spruce/Fir Group", 140: "Longleaf/Slash Pine Group",
    160: "Loblolly/Shortleaf Pine Group", 180: "Pinyon/Juniper Group", 200: "Douglas-fir Group",
    220: "Ponderosa Pine Group", 240: "Western White Pine Group", 260: "Fir/Spruce/Mountain Hemlock Group",
    280: "Lodgepole Pine Group", 300: "Hemlock/Sitka Spruce Group", 320: "Western Larch Group",
    340: "Redwood Group", 360: "Other Western Softwood Group", 370: "California Mixed Conifer Group",
    380: "Exotic Softwoods Group", 400: "Oak/Pine Group", 500: "Oak/Hickory Group",
    600: "Oak/Gum/Cypress Group", 700: "Elm/Ash/Cottonwood Group", 800: "Maple/Beech/Birch Group",
    900: "Aspen/Birch Group", 910: "Alder/Maple Group", 920: "Western Oak Group",
    940: "Tanoak/Laurel Group", 950: "Other Western Hardwoods Group", 980: "Tropical Hardwoods Group",
    990: "Exotic Hardwoods Group"
}

# --- Open raster once ---
with rasterio.open(raster_path) as src:
    pixel_width = src.transform[0]
    pixel_height = abs(src.transform[4])
    pixel_area_sq_m = pixel_width * pixel_height
    pixel_area_acres = pixel_area_sq_m * 0.000247105
    nodata = src.nodata
    raster_crs = src.crs

    # --- Function to reproject geometries ---
    def reproject_geom(geom, src_crs, dst_crs):
        project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform
        return mapping(transform(project, shape(geom)))

    # --- Loop through each shapefile ---
    for level, shapefile_path in shapefiles.items():
        print(f"\n🔍 Processing '{level}' pulpwood competition zone...")

        with fiona.open(shapefile_path, 'r') as shapefile:
            shp_crs = shapefile.crs
            total_zone_area = 0.0

            for feature in shapefile:
                polygon_id = feature.get("id", "unknown")
                geom = feature["geometry"]

                if shp_crs != raster_crs:
                    geom = reproject_geom(geom, shp_crs, raster_crs)

                try:
                    out_image, _ = mask(src, [geom], crop=True)
                except ValueError as e:
                    if "do not overlap" in str(e):
                        print(f"⚠️  Polygon {polygon_id} does not overlap the raster. Skipping...")
                        continue
                    else:
                        raise

                out_data = out_image[0]
                valid_data = out_data[out_data != nodata] if nodata is not None else out_data

                unique_types, pixel_counts = np.unique(valid_data, return_counts=True)

                forest_type_areas_acres = {
                    forest_type_mapping.get(ft, None): count * pixel_area_acres
                    for ft, count in zip(unique_types, pixel_counts)
                }
                forest_type_areas_acres = {
                    name: area for name, area in forest_type_areas_acres.items() if name is not None
                }

                total_zone_area += sum(forest_type_areas_acres.values())

            print(f"✅ Total forest area in '{level}' pulpwood zone: {total_zone_area:.2f} acres")



🔍 Processing 'low' pulpwood competition zone...
✅ Total forest area in 'low' pulpwood zone: 16360327.84 acres

🔍 Processing 'lowest' pulpwood competition zone...
✅ Total forest area in 'lowest' pulpwood zone: 1323309.05 acres
